# AI Studio Azure batch run Evaluation
### Chat Prompt Flow - All Data Run Base Run

Now in order to test these more thoroughly, we can use the Azure AI Studio to run batches of test data with the evaluation prompt flow on a larger dataset.

In [ ]:
import json
# Import required libraries
from promptflow.azure import PFClient
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from evaluate import run_azure_flow, run_azure_eval_flow

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

Populate the `config.json` file with the subscription_id, resource_group, and workspace_name.

In [ ]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Set the properties needed to run in Azure

In [ ]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
flow = "../contoso-chat"
data = "../data/alltestdata.jsonl"
run_name = "chat_all_data_base_run"
column_mapping={"customerId": "${data.customerId}","question": "${data.question}"}


Create a base run to use as the variant for the evaluation runs. 

In [ ]:
base_run = run_azure_flow(runtime, flow, run_name, data, column_mapping, pf_azure_client)

In [ ]:
pf_azure_client.stream(base_run)

In [ ]:
details = pf_azure_client.get_details(base_run)
details.head(10)

## Chat Prompt Flow Evaluation - All Data Eval Run

In [ ]:
eval_flow = "multi_flow/"
data = "../data/alltestdata.jsonl"
run_name = "chat_all_data_eval_run"
column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    }

In [ ]:
eval_run = run_azure_eval_flow(runtime, eval_flow, run_name, data, column_mapping, base_run, pf_azure_client)

In [ ]:
pf_azure_client.stream(eval_run)

In [ ]:
details = pf_azure_client.get_details(eval_run)
details.head(10)

In [ ]:

metrics = pf_azure_client.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf_azure_client.visualize([base_run, eval_run])

### Chat Prompt Flow - Chat Only Data Run Base Run


In [ ]:
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

flow = "../contoso-chat"
data = "../data/salestestdata.jsonl"
run_name = "chat_only_data_base_run"
column_mapping={"customerId": "${data.customerId}","question": "${data.question}"}

In [ ]:
base_run_chat_only = run_azure_flow(runtime, flow, run_name, data, column_mapping, pf_azure_client)

### Chat Prompt Flow - Chat Only Data Run Eval Run

In [ ]:
eval_flow = "multi_flow/"
run_name = "chat_only_data_eval_run"
column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    }

In [ ]:
eval_run = run_azure_eval_flow(runtime, eval_flow, run_name, data, column_mapping, base_run_chat_only, pf_azure_client)

In [ ]:
pf_azure_client.stream(eval_run)

In [ ]:
details = pf_azure_client.get_details(eval_run)
details.head(10)

In [ ]:
metrics = pf_azure_client.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))